# **(Title Change)**

## Objectives

* Write your notebook objective here, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write down which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [12]:
import os
current_dir = os.getcwd()
current_dir

'/Users/mahahussain/Desktop/Hackathon2'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [13]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [14]:
current_dir = os.getcwd()
current_dir

'/Users/mahahussain/Desktop'

# Section 1: ETL Process

### 1.1 Extract: Importing Libraries, Extracting & Describing the Dataset

- This section involves importing libraries necessary for subsequent data analysis and visualisation tasks.

These libraries are imported to handle data manipulation (pandas, numpy), create visualisations (seaborn, matplotlib, plotly), and perform statistical analysis (scipy.stats).

In [15]:
# Importing necessary libraries for data manipulation
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as st

We opted to access the CSV file, Online Retail.csv, directly from a GitHub repository using the raw URL of the file. 

By using this method, the CSV file is fetched directly from the GitHub repository without needing to download it manually, making the process efficient.

In [59]:
# Storing the url of the dataset and storing it in a DataFrame
url = "https://raw.githubusercontent.com/bvhadra/Online_Retail_Sales_Analysis/refs/heads/main/Online%20Retail.csv"
df = pd.read_csv(url) 

In [60]:
# Display the first few rows of the DataFrame to confirm successful import
print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55       17850  United Kingdom  
1  2010-12-01 08:26:00       3.39       17850  United Kingdom  
2  2010-12-01 08:26:00       2.75       17850  United Kingdom  
3  2010-12-01 08:26:00       3.39       17850  United Kingdom  
4  2010-12-01 08:26:00       3.39       17850  United Kingdom  


In [61]:
# Display the last few rows of the DataFrame to confirm successful import
print(df.tail())  

       InvoiceNo StockCode                      Description  Quantity  \
541904    581587     22613      PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899     CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254    CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255  CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138    BAKING SET 9 PIECE RETROSPOT          3   

                InvoiceDate  UnitPrice  CustomerID Country  
541904  2011-12-09 12:50:00       0.85       12680  France  
541905  2011-12-09 12:50:00       2.10       12680  France  
541906  2011-12-09 12:50:00       4.15       12680  France  
541907  2011-12-09 12:50:00       4.15       12680  France  
541908  2011-12-09 12:50:00       4.95       12680  France  


We then used the `df.describe()` function to generate a summary of key statistics for the numerical columns, helping us understand the data's distribution and identify potential outliers.

In [62]:
# Generates a summary of statistics (count, mean, std, min, max, etc.) for numerical columns in the DataFrame.
df[['Quantity', 'UnitPrice']].describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


#### Findings:

**Count**: Both **Quantity** and **UnitPrice** have 500k+ non-null entries, indicating a large dataset. Given our technical limitations, we will need to truncate this data.

1. **Mean**: The average **Quantity** is 9.55, and the average **UnitPrice** is 4.61, providing insight into typical values for these columns.

2. **Standard Deviation**: The standard deviations (218.08 for **Quantity** and 96.76 for **UnitPrice**) show significant variability in both columns, suggesting that the data includes a wide spread of values.

**Min/Max**: The extreme values are notable:
1. The **Quantity** column includes a minimum of **-80,995** and a maximum of **80,995**, with negative quantities being particularly concerning.

2. Similarly, the **UnitPrice** has a negative minimum of **-11,062.06** and a maximum of **38,970**, indicating the presence of abnormal negative values for prices.

We started by inspecting the counts of the null/not-null values in the columns using `df.count()` and `df.isna()` respectively.

In [63]:
# Display the count of non null values in each column
print("\nCount of non-NA values in each column:")
print(df.count())


Count of non-NA values in each column:
InvoiceNo      541909
StockCode      541909
Description    540455
Quantity       541909
InvoiceDate    541909
UnitPrice      541909
CustomerID     541909
Country        541909
dtype: int64


In [64]:
# Check for missing values in each column
print("\nMissing values in each column:")
print(df.isna().sum())


Missing values in each column:
InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64


The dataset has no missing values in most columns, except for **Description**, which contains 1,454 missing entries. 


#### Next Steps:

Following the extraction stage, we will be addressing the following key issues:

1. **Negative Values**: We need to look into the negative values found in both **Quantity** and **UnitPrice**, especially the extreme ones, as they could be errors or invalid entries. These might be caused by mistakes in data entry, returns, or system glitches.

2. **Clean the Data**: We will remove or correct the negative values in both columns, making sure only valid positive numbers are used in the analysis.

3. **Outlier Detection**: We’ll also need to deal with the extreme outliers in **Quantity** and **UnitPrice**. 

4. **Feature Engineering**: We will also perform feature engineering to ensure that the name column is properly standardised, especially since some items have the same name but come in different colours (e.g., "WHITE HANGING HEART T-LIGHT HOLDER" in different colorus). This will help avoid confusion and ensure consistency.


### 1.2 Transform: Data pre-processing & Feature Engineering

- This section involves cleaning the data, removing missing and invalid values. It also includes feature engineering and truncation.

Following the request from our stakeholders, we will focus exclusively on the United Kingdom (UK) for the analysis.

First, we want to confirm that the United Kingdom does not have any alternate names in the data set.

In [65]:
# Check unique values in the Country column
unique_countries = df['Country'].unique()
print("Unique values in 'Country' column:", unique_countries)

Unique values in 'Country' column: ['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


Having confirmed this, we will now filter the data accordingly.

In [67]:

# Filter the dataset to include only the United Kingdom
df_uk = df[df['Country'] == 'United Kingdom']

# Verify the filter by checking the unique countries again
unique_countries_after_filter = df_uk['Country'].unique()
print("Unique values in 'Country' column after filtering:", unique_countries_after_filter)


Unique values in 'Country' column after filtering: ['United Kingdom']


We will now begin to inspect the now filtered data.

In [72]:
# Display the first few rows of the DataFrame to get an overview.
print("First few rows of the data:")
print(df_uk.head())

First few rows of the data:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55       17850  United Kingdom  
1  2010-12-01 08:26:00       3.39       17850  United Kingdom  
2  2010-12-01 08:26:00       2.75       17850  United Kingdom  
3  2010-12-01 08:26:00       3.39       17850  United Kingdom  
4  2010-12-01 08:26:00       3.39       17850  United Kingdom  


In [75]:
# Summary statistics for numerical columns
print("\nSummary statistics for numerical columns:")
df_uk[['Quantity', 'UnitPrice']].describe()


Summary statistics for numerical columns:


,Quantity,UnitPrice
count,495478.000000,495478.000000
mean,8.605486,4.532422
std,227.588756,99.315438
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.100000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [73]:
# Display the count of non null values in each column
print("\nCount of non-NA values in each column:")
print(df_uk.count())


Count of non-NA values in each column:
InvoiceNo      495478
StockCode      495478
Description    494024
Quantity       495478
InvoiceDate    495478
UnitPrice      495478
CustomerID     495478
Country        495478
dtype: int64


In [74]:
print("\nMissing values in each column:")
print(df_uk.isna().sum())


Missing values in each column:
InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64


In terms of technical feasibility, truncating the dataset was a necessary decision. The full filtered dataset for the UK subset contains upwards of 400,000 records.

This reduction in size will allow us to focus on a representative sample of the data and ensure that the analysis remains efficient, whilst aligning with the stakeholder requirements.

In [77]:
df_truncated = df_uk.head(8000)

# Verifies the truncation
print(df_truncated.shape)
print(df_truncated.head())

(8000, 8)
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55       17850  United Kingdom  
1  2010-12-01 08:26:00       3.39       17850  United Kingdom  
2  2010-12-01 08:26:00       2.75       17850  United Kingdom  
3  2010-12-01 08:26:00       3.39       17850  United Kingdom  
4  2010-12-01 08:26:00       3.39       17850  United Kingdom  


# Section 2 : VISUALISATIONS

Section 2 content

In [17]:
print("test")

test


---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [18]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block after 'try' statement on line 2 (553063055.py, line 5)